In [1]:
import os
os.getcwd()

'/Users/charmay/Portfolio/qld_public_dental_list/notebooks'

In [3]:
from pathlib import Path
import pandas as pd

# Current working directory (notebooks folder)
BASE_DIR = Path.cwd()

# Move up one level to project root
PROJECT_ROOT = BASE_DIR.parent

# Build clean relative paths
monthly_path = PROJECT_ROOT / "data" / "processed" / "EDA" / "monthly_totals_waiting_and_treated.csv"
quarter_path = PROJECT_ROOT / "data" / "processed" / "EDA" / "pct_of_visit_type.csv"

df = pd.read_csv(monthly_path)
quarter_df = pd.read_csv(quarter_path)

df.head(10)

,date,total_waiting,total_treated
0,2020-02-29,145990,6701
1,2020-03-31,145432,4720
2,2020-07-31,156277,2958
3,2020-08-31,152711,4801
4,2020-09-30,152951,6551
5,2020-10-31,152532,5036
6,2020-11-30,151796,6001
7,2021-01-31,154478,3995
8,2021-02-28,153337,8978
9,2021-03-31,153112,9217


In [10]:
df.set_index('date', inplace=True)
df.index = pd.to_datetime(df.index)

# inplace=True modifies the original DataFrame directly. 
# If omitted, the method returns a new DataFrame and you 
# would need to assign it to a variable 
# (e.g., df_indexed = df.set_index('Date')).

In [11]:
df.head() 

,total_waiting,total_treated
date,,
2020-02-29,145990,6701
2020-03-31,145432,4720
2020-07-31,156277,2958
2020-08-31,152711,4801
2020-09-30,152951,6551


In [12]:
##Missing Months:
##                2020 --  1, 4-6, 12  - Q1 and Q4 quarter are partial. 
##                2021 --  7-12 - Q3 & Q4  
##                2022 --   1-6 - Q1 & Q2 
##                2023 --   1-3 - Q1

import plotly.graph_objects as go

fig = go.Figure()

# Add traces
fig.add_trace(go.Scatter(
    x=df['date'], 
    y=df['total_waiting'],
    mode='lines+markers',
    name='Patients Waiting'
))

# Shade each missing monthly period
fig.update_traces(connectgaps=False)

fig.add_vrect(x0='2019-09-01', x1='2020-01-30', 
              fillcolor='white', opacity=0.5, layer='below', line_width=0)
fig.add_vrect(x0='2020-04-01', x1='2020-06-30', 
              fillcolor='white', opacity=0.5, layer='below', line_width=0)
fig.add_vrect(x0='2020-12-01', x1='2020-12-30', 
              fillcolor='white', opacity=0.5, layer='below', line_width=0)

# 2021: Q3-Q4 missing
fig.add_vrect(x0='2021-07-01', x1='2021-12-31',
              fillcolor='white', opacity=0.5, layer='below', line_width=0,
              annotation_text='Missing Data', annotation_position='top left')

# 2022: Q1-Q2 missing  
fig.add_vrect(x0='2022-01-01', x1='2022-06-30',
              fillcolor='white', opacity=0.5, layer='below', line_width=0)

# 2023: Q1 missing
fig.add_vrect(x0='2023-01-01', x1='2023-03-31',
              fillcolor='white', opacity=0.5, layer='below', line_width=0)

fig.add_vline(x='2023-06-01', line_width=3, line_dash="dot", line_color="yellowgreen")
fig.add_annotation(x=pd.to_datetime('2023-06-01'),
                   y=1,
                   yref='paper',
                   text="Quarterly Collection Starts",
                   xanchor='left')

fig.update_layout(
    title='Snapshot of Patients Waiting List (Gaps Indicate Missing Data)',
    xaxis_title='',
    yaxis_title='Total Waiting',
    hovermode="x"
)
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.show()


KeyError: 'date'

Since April 2023, Patient Waitlist Numbers have inflated. How Does that compare to Patients treated?

Adding Patients treated as a secondary y-axis and the numbers have clashed visually - and secondary_y range must be matched to original y-axis using fig.update_yaxes()

Both y-Axis out of 100? for comparison - normalising to a baseline

In [ ]:
#index both series
df['waiting_indexed'] = df['total_waiting']/df['total_waiting'].iloc[0]*100 ##percentage difference between the current and first record
df['treated_indexed'] = df['total_treated']/df['total_treated'].iloc[0]*100

**Quarterly Timeseries**  
Trends in monthly numbers are consistant with quarterly numbers, collect the average of patients treated and waiting across months to inform their incomplete quarters. Complete missing quarters are redacted and highlighted to abide integrity.  

In [ ]:
#MoM Growth
df['waiting_growth'] = df['total_waiting'].pct_change()*100
df['treated_growth'] = df['total_treated'].pct_change()*100

In [ ]:
df_quarterly = df.resample('QE').mean()

In [ ]:
df_quarterly.head()

,total_waiting,total_treated
date,,
2020-03-31,145711.000000,5710.500000
2020-06-30,NaN,NaN
2020-09-30,153979.666667,4770.000000
2020-12-31,152164.000000,5518.500000
2021-03-31,153642.333333,7396.666667


In [ ]:
# visualise quarterly patients treated and waiting

##Missing Months:
##                2020 --  1, 4-6, 12  - Q1 and Q4 quarter are partial. 
##                2021 --  7-12 - Q3 & Q4  
##                2022 --   1-6 - Q1 & Q2 
##                2023 --   1-3 - Q1

import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])


# Add traces
fig.add_trace(go.Scatter(
    x=df_quarterly.index, 
    y=df_quarterly['total_waiting'],
    mode='lines+markers',
    name='Patients Waiting'
), secondary_y=False,)

fig.add_trace(go.Scatter(
    x=df_quarterly.index,
    y=df_quarterly['total_treated'],
    mode='lines+markers',
    name='Patients Treated'
), secondary_y=True,)

fig.update_yaxes(
    range=[0, df_quarterly['total_waiting'].max()*1.1],
    secondary_y=True
)
fig.update_yaxes(
    range=[0, df_quarterly['total_waiting'].max()*1.1],
    secondary_y=False
)
fig.update_traces(connectgaps=True)

# Shade each missing monthly period

fig.add_vrect(x0='2019-09-01', x1='2019-12-31', 
              fillcolor='white', opacity=0.5, layer='below', line_width=0)
fig.add_vrect(x0='2020-04-01', x1='2020-06-30', 
              fillcolor='white', opacity=0.5, layer='below', line_width=0)
# 2021: Q3-Q4 missing
fig.add_vrect(x0='2021-07-01', x1='2021-12-31',
              fillcolor='white', opacity=0.5, layer='below', line_width=0,
              annotation_text='Missing Data', annotation_position='top left')

# 2022: Q1-Q2 missing  
fig.add_vrect(x0='2022-01-01', x1='2022-06-30',
              fillcolor='white', opacity=0.5, layer='below', line_width=0)

# 2023: Q1 missing
fig.add_vrect(x0='2023-01-01', x1='2023-03-31',
              fillcolor='white', opacity=0.5, layer='below', line_width=0)

fig.add_vline(x='2023-06-01', line_width=3, line_dash="dot", line_color="yellowgreen")
fig.add_annotation(x=pd.to_datetime('2023-06-01'),
                   y=1,
                   yref='paper',
                   text="Quarterly Collection Starts",
                   xanchor='left')

fig.update_layout(
    title='Snapshot of Patients Waiting List (Gaps Indicate Missing Data)',
    xaxis_title='',
    yaxis_title='Total Waiting',
    hovermode="x"
)
fig.update_traces(mode="markers+lines", hovertemplate=None)
fig.show()


In [15]:
quarter_df.head(10)

,quarter_start,visit_type,total_waiting,pct_of_total_waiting
0,2020-01-01,Clinical Assessment,101,0.07
1,2020-01-01,General,127635,87.59
2,2020-01-01,General Anaesthetic Category 1,26,0.02
3,2020-01-01,General Anaesthetic Category 2,445,0.31
4,2020-01-01,General Anaesthetic Category 3,1812,1.24
5,2020-01-01,Priority 1,571,0.39
6,2020-01-01,Priority 2,4540,3.12
7,2020-01-01,Priority 3,10583,7.26
8,2020-07-01,Clinical Assessment,179,0.12
9,2020-07-01,General,133820,86.91


In [19]:
import plotly.express as px
fig = px.bar(quarter_df, x='quarter_start', y='pct_of_total_waiting', color='visit_type',
              title='Percentage of Visit Types Over Time')
fig.show()